In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import math

In [2]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

In [3]:
customers = pd.read_csv('customers.csv')
print(customers.info())
customers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23237 entries, 0 to 23236
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   WebsiteID         23237 non-null  int64 
 1   DeliveryPostCode  23183 non-null  object
 2   DeliveryCountry   23183 non-null  object
dtypes: int64(1), object(2)
memory usage: 544.7+ KB
None


,WebsiteID,DeliveryPostCode,DeliveryCountry
0,245,L39 2,United Kingdom
1,597,PR3 1,United Kingdom
2,739,CV2 2,United Kingdom
3,993,w1w 8,United Kingdom
4,994,LE15 8,United Kingdom
...,...,...,...
23232,975125,BD7 2,United Kingdom
23233,975126,MK40 4,United Kingdom
23234,975127,NW9 0,United Kingdom
23235,975129,GL1 4,United Kingdom


In [4]:
orders = pd.read_csv('orders.csv')
print(orders.info())
orders

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32063 entries, 0 to 32062
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   OrderID            32063 non-null  int64  
 1   WebsiteID          32063 non-null  int64  
 2   OrderDate          32063 non-null  object 
 3   StatusName         32063 non-null  object 
 4   StatusId           32063 non-null  int64  
 5   PaymentOptionName  32063 non-null  object 
 6   PaymentType        32063 non-null  int64  
 7   DespatchedCourier  32063 non-null  object 
 8   ProductsExVAT      32063 non-null  float64
 9   DeliveryExVAT      32063 non-null  float64
 10  VAT                32063 non-null  float64
 11  SubTotal           32063 non-null  float64
 12  GrandTotal         32063 non-null  float64
 13  DateDespatched     31006 non-null  object 
 14  DeliveryCost       31131 non-null  float64
dtypes: float64(6), int64(4), object(5)
memory usage: 3.7+ MB
None


,OrderID,WebsiteID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost
0,2729772,971172,03/06/2024 00:15:07,Dispatched,6,Paypal,1,DX Express,26.46,7.37,6.76,26.46,40.59,03/06/2024 00:00:00,6.95
1,2729773,971173,03/06/2024 00:33:45,Dispatched,6,Payment Gateway,2,DX Express,18.80,7.37,5.23,18.80,31.40,03/06/2024 00:00:00,6.95
2,2729774,889210,03/06/2024 00:52:11,Dispatched,6,Payment Gateway,2,DX Freight,351.07,19.99,74.22,351.07,445.28,03/06/2024 00:00:00,25.33
3,2729775,971174,03/06/2024 00:54:54,Dispatched,6,Payment Gateway,2,DX Express,56.14,0.00,11.23,56.14,67.37,03/06/2024 00:00:00,6.95
4,2729776,971175,03/06/2024 01:19:05,Dispatched,6,Paypal,1,DX Freight,144.16,0.00,28.84,144.16,173.00,03/06/2024 00:00:00,16.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32058,2761830,948075,30/06/2024 23:43:02,Dispatched,6,Payment Gateway,2,DX Express,85.42,0.00,17.09,85.42,102.51,01/07/2024 00:00:00,7.82
32059,2761831,709642,30/06/2024 23:43:29,Dispatched,6,Payment Gateway,2,DX Freight,63.84,0.00,12.77,63.84,76.61,01/07/2024 00:00:00,11.07
32060,2761832,953130,30/06/2024 23:46:06,Dispatched,6,Paypal,1,DX Freight,51.35,0.00,10.27,51.35,61.62,01/07/2024 00:00:00,13.06
32061,2761833,600960,30/06/2024 23:50:55,Dispatched,6,Paypal,1,DX Express,19.49,7.26,5.35,19.49,32.10,01/07/2024 00:00:00,6.85


In [5]:
order_details = pd.read_csv('order_details.csv')
print(order_details.info())
order_details

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76573 entries, 0 to 76572
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OrderID         76573 non-null  int64  
 1   PartNumber      76573 non-null  object 
 2   Description     76573 non-null  object 
 3   Packs           76573 non-null  int64  
 4   PackPrice       76573 non-null  float64
 5   UnitsPerPack    76573 non-null  int64  
 6   PackWeight      76573 non-null  float64
 7   LineWeight      76402 non-null  float64
 8   PalletQuantity  76573 non-null  int64  
 9   CubicMPerUnit   72449 non-null  float64
dtypes: float64(4), int64(4), object(2)
memory usage: 5.8+ MB
None


,OrderID,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit
0,2729772,BB305X435,Clear bubble bags 305x435mm - pack of 150,1,26.46,150,2.55,2.55,2400,0.000925
1,2729773,PBL1215,Light duty polythene bags - 11.75x14.75 inches...,1,18.80,1000,4.40,4.40,180000,0.000006
2,2729774,CSW222X150X88-BPB6,Brown postal boxes - 222x150x88mm (8.75x6x3.5 ...,1,20.55,50,3.90,3.90,2200,0.000726
3,2729774,PS320X140X420-BROWN,Twist handle paper carrier bags brown 320x140x...,1,52.28,250,5.00,5.00,3750,0.000144
4,2729774,TPK048050S,"Paper kraft tape - 48mm x 50m, Solvent - pack ...",3,13.05,6,1.28,3.84,2592,0.000677
...,...,...,...,...,...,...,...,...,...,...
76568,2761831,PTAF1828E,Ream of acid free tissue paper - 480 sheets - ...,6,6.69,1,2.72,16.32,105,0.009598
76569,2761831,VF15EF,Starch based loose fill - 15 cu ft bag,1,23.70,1,2.00,2.00,4,0.376712
76570,2761832,CDW159.7505,Double wall boxes - 15x9.75x5 inches (381x248x...,5,10.27,20,4.93,24.65,600,0.002269
76571,2761833,TUBE76.6X625X2.5EC,Cardboard postal tube - 625mmx76mmx2.5mm - pac...,1,19.49,20,6.60,6.60,300,0.004359


In [6]:
kite1 = pd.merge(customers, orders, how='inner', on = 'WebsiteID')
kite1

,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost
0,245,L39 2,United Kingdom,2732430,04/06/2024 12:47:57,Dispatched,6,On Account,3,DX Freight,82.45,0.0,16.49,82.45,98.94,04/06/2024 00:00:00,11.17
1,597,PR3 1,United Kingdom,2737394,07/06/2024 14:05:01,Dispatched,6,Payment Gateway,2,MFS,190.19,0.0,38.04,190.19,228.23,07/06/2024 00:00:00,39.43
2,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88
3,993,w1w 8,United Kingdom,2747001,17/06/2024 12:28:36,Dispatched,6,Sample,4,DHLExpress,0.00,0.0,0.00,0.00,0.00,18/06/2024 00:00:00,6.88
4,993,w1w 8,United Kingdom,2747068,17/06/2024 12:53:39,Dispatched,6,Sample,4,DHLExpress,0.00,0.0,0.00,0.00,0.00,18/06/2024 00:00:00,6.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32058,975125,BD7 2,United Kingdom,2761820,30/06/2024 23:03:16,Dispatched,6,Payment Gateway,2,DX Express,45.14,0.0,9.03,45.14,54.17,01/07/2024 00:00:00,6.85
32059,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85
32060,975127,NW9 0,United Kingdom,2761818,30/06/2024 22:59:39,Dispatched,6,Payment Gateway,2,DX Freight,68.58,0.0,13.71,68.58,82.29,01/07/2024 00:00:00,12.76
32061,975129,GL1 4,United Kingdom,2761824,30/06/2024 23:16:40,Dispatched,6,Payment Gateway,2,DX Freight,30.41,12.4,8.56,30.41,51.37,01/07/2024 00:00:00,11.07


In [7]:
kite2 = pd.merge(kite1, order_details, how='inner', on = 'OrderID')
print(kite2.info())
kite2.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76573 entries, 0 to 76572
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   WebsiteID          76573 non-null  int64  
 1   DeliveryPostCode   76380 non-null  object 
 2   DeliveryCountry    76380 non-null  object 
 3   OrderID            76573 non-null  int64  
 4   OrderDate          76573 non-null  object 
 5   StatusName         76573 non-null  object 
 6   StatusId           76573 non-null  int64  
 7   PaymentOptionName  76573 non-null  object 
 8   PaymentType        76573 non-null  int64  
 9   DespatchedCourier  76573 non-null  object 
 10  ProductsExVAT      76573 non-null  float64
 11  DeliveryExVAT      76573 non-null  float64
 12  VAT                76573 non-null  float64
 13  SubTotal           76573 non-null  float64
 14  GrandTotal         76573 non-null  float64
 15  DateDespatched     74083 non-null  object 
 16  DeliveryCost       744

,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit
0,245,L39 2,United Kingdom,2732430,04/06/2024 12:47:57,Dispatched,6,On Account,3,DX Freight,82.45,0.00,16.49,82.45,98.94,04/06/2024 00:00:00,11.17,BSE2X750X100-ET,Small bubble wrap - Kite premium easy tear - 7...,2,27.20,1,6.30,12.60,4,0.317400
1,245,L39 2,United Kingdom,2732430,04/06/2024 12:47:57,Dispatched,6,On Account,3,DX Freight,82.45,0.00,16.49,82.45,98.94,04/06/2024 00:00:00,11.17,CDW090606M,Double wall boxes - 9x6x6 inches (229x152x152 ...,1,13.97,20,2.46,2.46,1080,0.001091
2,245,L39 2,United Kingdom,2732430,04/06/2024 12:47:57,Dispatched,6,On Account,3,DX Freight,82.45,0.00,16.49,82.45,98.94,04/06/2024 00:00:00,11.17,CDW120906M,Double wall boxes - 12x9x6 inches (305x229x152...,1,14.08,20,4.58,4.58,720,0.001944
3,597,PR3 1,United Kingdom,2737394,07/06/2024 14:05:01,Dispatched,6,Payment Gateway,2,MFS,190.19,0.00,38.04,190.19,228.23,07/06/2024 00:00:00,39.43,CDW241818M-REV1,Double wall boxes - 24x18x18 inches (610x457x4...,7,27.17,20,22.50,157.50,140,0.009052
4,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.00,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR7.5X7.5-KITE,Plain grip seal bags - 7.5x7.5 inches (191x191...,3,6.44,100,0.33,0.99,360000,0.000007
5,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.00,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR4X5.5-KITE,Plain grip seal bags - 4x5.5 inches (102x140 m...,3,3.93,100,0.14,0.42,648000,0.000004
6,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.00,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR12.75X12.75-KITE,Plain grip seal bags - 12.75x12.75 inches (324...,3,10.35,100,0.87,2.61,120000,0.000019
7,993,w1w 8,United Kingdom,2747001,17/06/2024 12:28:36,Dispatched,6,Sample,4,DHLExpress,0.00,0.00,0.00,0.00,0.00,18/06/2024 00:00:00,6.88,AS-1BOTTLE-SET,SAMPLE Air shock - 1 bottle pack,1,0.00,1,0.14,0.14,0,0.002716
8,993,w1w 8,United Kingdom,2747068,17/06/2024 12:53:39,Dispatched,6,Sample,4,DHLExpress,0.00,0.00,0.00,0.00,0.00,18/06/2024 00:00:00,6.88,PA-FLEX-1BOT,SAMPLE - Flexi-Hex sleeve - 325 x 35 x 20mm,1,0.00,1,0.14,0.14,0,0.000288
9,993,w1w 8,United Kingdom,2747991,18/06/2024 08:20:40,Deleted,7,Sample,4,DHLExpress,0.00,0.00,0.00,0.00,0.00,NaN,NaN,PA-FLEX-1BOT,SAMPLE - Flexi-Hex sleeve - 325 x 35 x 20mm,1,0.00,1,0.14,0.14,0,0.000288


In [8]:
kite2.isna().sum()

WebsiteID               0
DeliveryPostCode      193
DeliveryCountry       193
OrderID                 0
OrderDate               0
StatusName              0
StatusId                0
PaymentOptionName       0
PaymentType             0
DespatchedCourier       0
ProductsExVAT           0
DeliveryExVAT           0
VAT                     0
SubTotal                0
GrandTotal              0
DateDespatched       2490
DeliveryCost         2139
PartNumber              0
Description             0
Packs                   0
PackPrice               0
UnitsPerPack            0
PackWeight              0
LineWeight            171
PalletQuantity          0
CubicMPerUnit        4124
dtype: int64

In [9]:
kite2['StatusName'].value_counts()

StatusName
Dispatched    74390
Deleted        2182
Processing        1
Name: count, dtype: int64

In [10]:
kite2 = kite2[kite2['StatusName'] == 'Dispatched']
kite2['StatusName'].value_counts()

StatusName
Dispatched    74390
Name: count, dtype: int64

In [11]:
kite2.isna().sum()

WebsiteID               0
DeliveryPostCode      181
DeliveryCountry       181
OrderID                 0
OrderDate               0
StatusName              0
StatusId                0
PaymentOptionName       0
PaymentType             0
DespatchedCourier       0
ProductsExVAT           0
DeliveryExVAT           0
VAT                     0
SubTotal                0
GrandTotal              0
DateDespatched        316
DeliveryCost          314
PartNumber              0
Description             0
Packs                   0
PackPrice               0
UnitsPerPack            0
PackWeight              0
LineWeight            160
PalletQuantity          0
CubicMPerUnit        3979
dtype: int64

In [12]:
# function for extracting the alphabetic part of postcodes
def PostCode(x):
    a = ''.join([i for i in x if not i.isdigit()])
    b = re.sub('[^\w]','', a)
    return b.upper()

In [13]:
kite2['PostCode'] = kite2['DeliveryPostCode'].astype(str)
kite2['PostCode'] = kite2['PostCode'].apply(PostCode)
kite2['PostCode'].value_counts()

PostCode
B     2378
LE    1796
LS    1615
NG    1462
S     1425
      ... 
WJ       4
GD       2
GY       2
C        1
ZE       1
Name: count, Length: 158, dtype: int64

In [14]:
# fixing 0X postcode
kite2.loc[kite2['PostCode']=='X','PostCode'] = 'OX'
(kite2['PostCode']=='X').sum()

0

In [15]:
# list of postcodes within 50 miles delivery radius (excluded GL)
delivery_postcode = ['WS','WV','DY','WR','OX','MK','NN','LE','DE','B','CV','NG']

In [16]:
kite3 = kite2.loc[kite2['PostCode'].isin(delivery_postcode),:].reset_index()
kite3

,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode
0,4,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR7.5X7.5-KITE,Plain grip seal bags - 7.5x7.5 inches (191x191...,3,6.44,100,0.33,0.99,360000,0.000007,CV
1,5,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR4X5.5-KITE,Plain grip seal bags - 4x5.5 inches (102x140 m...,3,3.93,100,0.14,0.42,648000,0.000004,CV
2,6,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR12.75X12.75-KITE,Plain grip seal bags - 12.75x12.75 inches (324...,3,10.35,100,0.87,2.61,120000,0.000019,CV
3,10,994,LE15 8,United Kingdom,2754306,24/06/2024 10:12:17,Dispatched,6,On Account,3,DX Express,77.00,0.0,15.40,77.00,92.40,24/06/2024 00:00:00,6.88,CSW151010M,Single wall cardboard boxes - 15x10x10 inches ...,4,19.25,25,4.81,19.24,900,0.001700,LE
4,21,3215,DY5 1,United Kingdom,2750321,19/06/2024 12:39:42,Dispatched,6,On Account,3,DX Freight,133.54,0.0,26.70,133.54,160.24,19/06/2024 00:00:00,18.64,CDW181212M-REV1,Double wall boxes - 18x12x12 inches (457x305x3...,1,23.21,20,10.30,10.30,560,0.004439,DY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12900,76539,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,PJET-SPEEDMAN-BOX,Speedman Box - Protective Paper Void Fill,1,38.93,1,12.52,12.52,24,0.033640,LE
12901,76540,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,CSW090604,Single wall cardboard boxes - 9x6x4 inches (22...,4,4.98,30,1.80,7.20,3600,0.000415,LE
12902,76559,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,CSW101X101X19-LLPBR5,Premium large letter postal box 101 x 101 x 19...,1,19.14,150,2.23,2.23,10800,0.000059,MK
12903,76560,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,PA-SH-NAT-4KG-B,ShredPak Natural paper 4Kg bale,1,21.50,1,4.00,4.00,40,0.033792,MK


In [17]:
# mapping the postcodes onto 6 distinct delivery routes, 0 is for Coventry area only.
kite3['PostCode_Label'] = kite3['PostCode'].map({'B':1,'DY':1,'WV':1,'WS':1,'LE':2,'DE':2,'NG':2,
                                                'NN':3,'MK':3,'OX':4,'WR':5,'CV':0})
kite3

,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode,PostCode_Label
0,4,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR7.5X7.5-KITE,Plain grip seal bags - 7.5x7.5 inches (191x191...,3,6.44,100,0.33,0.99,360000,0.000007,CV,0
1,5,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR4X5.5-KITE,Plain grip seal bags - 4x5.5 inches (102x140 m...,3,3.93,100,0.14,0.42,648000,0.000004,CV,0
2,6,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR12.75X12.75-KITE,Plain grip seal bags - 12.75x12.75 inches (324...,3,10.35,100,0.87,2.61,120000,0.000019,CV,0
3,10,994,LE15 8,United Kingdom,2754306,24/06/2024 10:12:17,Dispatched,6,On Account,3,DX Express,77.00,0.0,15.40,77.00,92.40,24/06/2024 00:00:00,6.88,CSW151010M,Single wall cardboard boxes - 15x10x10 inches ...,4,19.25,25,4.81,19.24,900,0.001700,LE,2
4,21,3215,DY5 1,United Kingdom,2750321,19/06/2024 12:39:42,Dispatched,6,On Account,3,DX Freight,133.54,0.0,26.70,133.54,160.24,19/06/2024 00:00:00,18.64,CDW181212M-REV1,Double wall boxes - 18x12x12 inches (457x305x3...,1,23.21,20,10.30,10.30,560,0.004439,DY,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12900,76539,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,PJET-SPEEDMAN-BOX,Speedman Box - Protective Paper Void Fill,1,38.93,1,12.52,12.52,24,0.033640,LE,2
12901,76540,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,CSW090604,Single wall cardboard boxes - 9x6x4 inches (22...,4,4.98,30,1.80,7.20,3600,0.000415,LE,2
12902,76559,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,CSW101X101X19-LLPBR5,Premium large letter postal box 101 x 101 x 19...,1,19.14,150,2.23,2.23,10800,0.000059,MK,3
12903,76560,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,PA-SH-NAT-4KG-B,ShredPak Natural paper 4Kg bale,1,21.50,1,4.00,4.00,40,0.033792,MK,3


In [18]:
kite3.isna().sum()

index                  0
WebsiteID              0
DeliveryPostCode       0
DeliveryCountry        0
OrderID                0
OrderDate              0
StatusName             0
StatusId               0
PaymentOptionName      0
PaymentType            0
DespatchedCourier      0
ProductsExVAT          0
DeliveryExVAT          0
VAT                    0
SubTotal               0
GrandTotal             0
DateDespatched        51
DeliveryCost          51
PartNumber             0
Description            0
Packs                  0
PackPrice              0
UnitsPerPack           0
PackWeight             0
LineWeight            22
PalletQuantity         0
CubicMPerUnit        690
PostCode               0
PostCode_Label         0
dtype: int64

In [19]:
# excluded direct deliveries
kite3 = kite3[kite3['DeliveryCost'] != 0]

In [20]:
# excluded null delivery costs
kite3.dropna(subset = 'DeliveryCost', inplace = True)

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/3949600830.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3.dropna(subset = 'DeliveryCost', inplace = True)


In [21]:
kite3.isna().sum()

index                  0
WebsiteID              0
DeliveryPostCode       0
DeliveryCountry        0
OrderID                0
OrderDate              0
StatusName             0
StatusId               0
PaymentOptionName      0
PaymentType            0
DespatchedCourier      0
ProductsExVAT          0
DeliveryExVAT          0
VAT                    0
SubTotal               0
GrandTotal             0
DateDespatched         0
DeliveryCost           0
PartNumber             0
Description            0
Packs                  0
PackPrice              0
UnitsPerPack           0
PackWeight             0
LineWeight            10
PalletQuantity         0
CubicMPerUnit        650
PostCode               0
PostCode_Label         0
dtype: int64

In [22]:
# change to datetime format without time component
def date(x):
    y = x.strftime('%Y-%m-%d')
    return pd.to_datetime(y)

In [23]:
kite3['Order_Date'] = pd.to_datetime(kite3['OrderDate'], dayfirst = True).apply(date)
kite3['Date_Despatched'] = pd.to_datetime(kite3['DateDespatched'], dayfirst = True).apply(date)
kite3['Despatch_Delay'] = kite3['Date_Despatched'] - kite3['Order_Date']

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/2502413404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Order_Date'] = pd.to_datetime(kite3['OrderDate'], dayfirst = True).apply(date)
/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/2502413404.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Date_Despatched'] = pd.to_datetime(kite3['DateDespatched'], dayfirst = True).apply(date)
/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/250241340

In [24]:
# determine the day of week the orders are made
def weekday(x):
    y = x.strftime('%A')
    return y

kite3['DayOfWeek'] = kite3['Order_Date'].apply(weekday)
kite3['DayOfWeek_n'] = kite3['DayOfWeek'].map({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})
kite3

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/3061641742.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['DayOfWeek'] = kite3['Order_Date'].apply(weekday)
/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/3061641742.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['DayOfWeek_n'] = kite3['DayOfWeek'].map({'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7})


,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode,PostCode_Label,Order_Date,Date_Despatched,Despatch_Delay,DayOfWeek,DayOfWeek_n
0,4,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR7.5X7.5-KITE,Plain grip seal bags - 7.5x7.5 inches (191x191...,3,6.44,100,0.33,0.99,360000,0.000007,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2
1,5,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR4X5.5-KITE,Plain grip seal bags - 4x5.5 inches (102x140 m...,3,3.93,100,0.14,0.42,648000,0.000004,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2
2,6,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR12.75X12.75-KITE,Plain grip seal bags - 12.75x12.75 inches (324...,3,10.35,100,0.87,2.61,120000,0.000019,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2
3,10,994,LE15 8,United Kingdom,2754306,24/06/2024 10:12:17,Dispatched,6,On Account,3,DX Express,77.00,0.0,15.40,77.00,92.40,24/06/2024 00:00:00,6.88,CSW151010M,Single wall cardboard boxes - 15x10x10 inches ...,4,19.25,25,4.81,19.24,900,0.001700,LE,2,2024-06-24,2024-06-24,0 days,Monday,1
4,21,3215,DY5 1,United Kingdom,2750321,19/06/2024 12:39:42,Dispatched,6,On Account,3,DX Freight,133.54,0.0,26.70,133.54,160.24,19/06/2024 00:00:00,18.64,CDW181212M-REV1,Double wall boxes - 18x12x12 inches (457x305x3...,1,23.21,20,10.30,10.30,560,0.004439,DY,1,2024-06-19,2024-06-19,0 days,Wednesday,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12900,76539,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,PJET-SPEEDMAN-BOX,Speedman Box - Protective Paper Void Fill,1,38.93,1,12.52,12.52,24,0.033640,LE,2,2024-06-30,2024-07-01,1 days,Sunday,7
12901,76540,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,CSW090604,Single wall cardboard boxes - 9x6x4 inches (22...,4,4.98,30,1.80,7.20,3600,0.000415,LE,2,2024-06-30,2024-07-01,1 days,Sunday,7
12902,76559,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,CSW101X101X19-LLPBR5,Premium large letter postal box 101 x 101 x 19...,1,19.14,150,2.23,2.23,10800,0.000059,MK,3,2024-06-30,2024-07-01,1 days,Sunday,7
12903,76560,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,PA-SH-NAT-4KG-B,ShredPak Natural paper 4Kg bale,1,21.50,1,4.00,4.00,40,0.033792,MK,3,2024-06-30,2024-07-01,1 days,Sunday,7


In [25]:
kite3['DeliveryExVAT'].value_counts()

DeliveryExVAT
0.00     10937
19.99     1089
7.29       341
7.30       157
7.26       125
25.00       35
7.37        33
5.95        24
12.51       20
12.45       14
27.28        9
12.40        8
27.25        7
9.66         3
27.29        2
32.44        2
60.45        1
7.63         1
8.34         1
8.32         1
25.94        1
Name: count, dtype: int64

In [26]:
# determine the expected delay between order and delivery dates by considering the type of delivery and the day of week
# when the orders are made, 2-4 working days are assumed to be 2 days

kite3.loc[((kite3['DeliveryExVAT'] == 19.99) | (kite3['DeliveryExVAT'] > 25)) & (kite3['DayOfWeek_n']<=4), 'Expected_Delivery'] = 1
kite3.loc[((kite3['DeliveryExVAT'] == 19.99) | (kite3['DeliveryExVAT'] > 25)) & (kite3['DayOfWeek_n']>=5), 'Expected_Delivery'] = 8-kite3['DayOfWeek_n']

kite3.loc[kite3['DeliveryExVAT'] == 25, 'Expected_Delivery'] = 6 - kite3['DayOfWeek_n']

kite3.loc[(kite3['DeliveryExVAT'] < 25) & (kite3['DeliveryExVAT'] != 19.99) & (kite3['DayOfWeek_n']<=3), 'Expected_Delivery'] = 2
kite3.loc[(kite3['DeliveryExVAT'] < 25) & (kite3['DeliveryExVAT'] != 19.99) & (kite3['DayOfWeek_n']==4), 'Expected_Delivery'] = 4
kite3.loc[(kite3['DeliveryExVAT'] < 25) & (kite3['DeliveryExVAT'] != 19.99) & (kite3['DayOfWeek_n']>4), 'Expected_Delivery'] = 9-kite3['DayOfWeek_n']


kite3['Expected_Delivery'] = kite3['Expected_Delivery'].astype(int)

kite3

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/3618103927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3.loc[((kite3['DeliveryExVAT'] == 19.99) | (kite3['DeliveryExVAT'] > 25)) & (kite3['DayOfWeek_n']<=4), 'Expected_Delivery'] = 1
/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/3618103927.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Expected_Delivery'] = kite3['Expected_Delivery'].astype(int)


,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode,PostCode_Label,Order_Date,Date_Despatched,Despatch_Delay,DayOfWeek,DayOfWeek_n,Expected_Delivery
0,4,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR7.5X7.5-KITE,Plain grip seal bags - 7.5x7.5 inches (191x191...,3,6.44,100,0.33,0.99,360000,0.000007,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2
1,5,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR4X5.5-KITE,Plain grip seal bags - 4x5.5 inches (102x140 m...,3,3.93,100,0.14,0.42,648000,0.000004,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2
2,6,739,CV2 2,United Kingdom,2748523,18/06/2024 11:40:50,Dispatched,6,Payment Gateway,2,DX Express,62.16,0.0,12.43,62.16,74.59,18/06/2024 00:00:00,6.88,PBR12.75X12.75-KITE,Plain grip seal bags - 12.75x12.75 inches (324...,3,10.35,100,0.87,2.61,120000,0.000019,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2
3,10,994,LE15 8,United Kingdom,2754306,24/06/2024 10:12:17,Dispatched,6,On Account,3,DX Express,77.00,0.0,15.40,77.00,92.40,24/06/2024 00:00:00,6.88,CSW151010M,Single wall cardboard boxes - 15x10x10 inches ...,4,19.25,25,4.81,19.24,900,0.001700,LE,2,2024-06-24,2024-06-24,0 days,Monday,1,2
4,21,3215,DY5 1,United Kingdom,2750321,19/06/2024 12:39:42,Dispatched,6,On Account,3,DX Freight,133.54,0.0,26.70,133.54,160.24,19/06/2024 00:00:00,18.64,CDW181212M-REV1,Double wall boxes - 18x12x12 inches (457x305x3...,1,23.21,20,10.30,10.30,560,0.004439,DY,1,2024-06-19,2024-06-19,0 days,Wednesday,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12900,76539,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,PJET-SPEEDMAN-BOX,Speedman Box - Protective Paper Void Fill,1,38.93,1,12.52,12.52,24,0.033640,LE,2,2024-06-30,2024-07-01,1 days,Sunday,7,2
12901,76540,975118,LE4 8,United Kingdom,2761778,30/06/2024 21:18:19,Dispatched,6,Payment Gateway,2,DX Express,62.18,0.0,12.44,62.18,74.62,01/07/2024 00:00:00,6.85,CSW090604,Single wall cardboard boxes - 9x6x4 inches (22...,4,4.98,30,1.80,7.20,3600,0.000415,LE,2,2024-06-30,2024-07-01,1 days,Sunday,7,2
12902,76559,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,CSW101X101X19-LLPBR5,Premium large letter postal box 101 x 101 x 19...,1,19.14,150,2.23,2.23,10800,0.000059,MK,3,2024-06-30,2024-07-01,1 days,Sunday,7,2
12903,76560,975126,MK40 4,United Kingdom,2761810,30/06/2024 22:34:32,Dispatched,6,Payment Gateway,2,DX Express,54.89,0.0,10.98,54.89,65.87,01/07/2024 00:00:00,6.85,PA-SH-NAT-4KG-B,ShredPak Natural paper 4Kg bale,1,21.50,1,4.00,4.00,40,0.033792,MK,3,2024-06-30,2024-07-01,1 days,Sunday,7,2


In [27]:
kite3['Despatch_Delay'] = kite3['Despatch_Delay'].astype(str)

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/2947874870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Despatch_Delay'] = kite3['Despatch_Delay'].astype(str)


In [28]:
# convert despatch delay into numbers
def number_only(x):
    a = ''.join(i for i in x if i.isdigit())
    return a

In [29]:
kite3['Despatch_Delay_n'] = kite3['Despatch_Delay'].apply(number_only)
kite3['Despatch_Delay_n'] = kite3['Despatch_Delay_n'].astype(int)

/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/4045158129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Despatch_Delay_n'] = kite3['Despatch_Delay'].apply(number_only)
/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/4045158129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3['Despatch_Delay_n'] = kite3['Despatch_Delay_n'].astype(int)


In [30]:
kite3.reset_index(inplace=True)

In [31]:
# determine the delivery date for each order

index = range(len(kite3))

for i in index:
    if kite3['Despatch_Delay_n'][i] >= kite3['Expected_Delivery'][i]:
        kite3.loc[i,'Delivery_Date'] = kite3['Date_Despatched'][i]
    else:
        kite3.loc[i,'Delivery_Date'] = kite3['Order_Date'][i] + datetime.timedelta(days = kite3['Expected_Delivery'][i].astype(float))


/var/folders/bj/b6dk33hs5cx_fj291gkmt_dh0000gr/T/ipykernel_72138/131206475.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kite3.loc[i,'Delivery_Date'] = kite3['Order_Date'][i] + datetime.timedelta(days = kite3['Expected_Delivery'][i].astype(float))


In [32]:
# some of the orders are delayed into July
pd.DataFrame(kite3['Delivery_Date'].value_counts()).sort_values('Delivery_Date')

,count
Delivery_Date,
2024-06-04,47
2024-06-05,813
2024-06-06,710
2024-06-07,724
2024-06-08,1
2024-06-10,532
2024-06-11,619
2024-06-12,657
2024-06-13,623


In [33]:
kite3 = kite3.loc[kite3['Delivery_Date'] <= '2024-06-30',:]

In [34]:
kite3[(kite3['PalletQuantity']==0) & (kite3['Description'].str.contains('Custom boxes'))]

,level_0,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode,PostCode_Label,Order_Date,Date_Despatched,Despatch_Delay,DayOfWeek,DayOfWeek_n,Expected_Delivery,Despatch_Delay_n,Delivery_Date
1052,1059,6022,181941,B19 3,United Kingdom,2732830,04/06/2024 14:54:08,Dispatched,6,On Account,3,MFS,3500.00,0.0,700.00,3500.00,4200.00,25/06/2024 00:00:00,714.58,CDW400X550X150-82,Custom boxes - 400x550x150mm 0201 BC125WT Whit...,2000,1.75,1,0.74,1480.0,0,NaN,B,1,2024-06-04,2024-06-25,21 days,Tuesday,2,2,21,2024-06-25
4743,4779,26615,626869,OX29 8,United Kingdom,2734771,05/06/2024 16:00:39,Dispatched,6,"Cheque, BACS or pro-forma",0,MFS,1058.40,0.0,211.68,1058.40,1270.08,21/06/2024 00:00:00,79.11,CSW240X125X315-62,Custom boxes - 240x125x315mm 0201 B200KT brown...,2700,0.39,1,0.15,405.0,0,NaN,OX,4,2024-06-05,2024-06-21,16 days,Wednesday,3,2,16,2024-06-21
5065,5110,28727,642711,LE17 4,United Kingdom,2732776,04/06/2024 14:39:26,Dispatched,6,On Account,3,MFS,1152.00,0.0,230.40,1152.00,1382.40,14/06/2024 00:00:00,77.24,CSW430X220X150-62,Custom boxes - 430x220x150mm 0201 B200KT brown...,2000,0.58,1,0.22,440.0,0,NaN,LE,2,2024-06-04,2024-06-14,10 days,Tuesday,2,2,10,2024-06-14
5983,6033,34672,685742,B8 3,United Kingdom,2735051,05/06/2024 23:05:39,Dispatched,6,Payment Gateway,2,MFS,1562.50,0.0,312.50,1562.50,1875.00,21/06/2024 00:00:00,324.81,CDW600X155X150-72,Custom boxes - 600x155x150mm 0201 BC125KT Brown,2500,0.63,1,0.26,650.0,0,NaN,B,1,2024-06-05,2024-06-21,16 days,Wednesday,3,2,16,2024-06-21
6539,6593,37512,709493,CV21 2,United Kingdom,2743009,12/06/2024 16:42:04,Dispatched,6,Paypal,1,MFS,700.48,0.0,140.10,700.48,840.58,20/06/2024 00:00:00,98.45,CDW620X400X590-72,Custom boxes - 620x400x590mm 0201 BC125KT Brown,160,4.38,1,1.12,179.2,0,NaN,CV,0,2024-06-12,2024-06-20,8 days,Wednesday,3,2,8,2024-06-20


In [35]:
# filling the pallet quantity for custom boxes orders, based on values for similar sized boxes
kite3.loc[kite3['PartNumber'] == 'CSW430X220X150-62','PalletQuantity'] = 200
kite3.loc[kite3['PartNumber'] == 'CDW400X550X150-82','PalletQuantity'] = 140
kite3.loc[kite3['PartNumber'] == 'CSW240X125X315-62','PalletQuantity'] = 720
kite3.loc[kite3['PartNumber'] == 'CDW600X155X150-72','PalletQuantity'] = 675
kite3.loc[kite3['PartNumber'] == 'CDW620X400X590-72','PalletQuantity'] = 100
kite3.loc[kite3['PartNumber'] == 'CSW650X170X650-32','PalletQuantity'] = 100

In [36]:
len(kite3[kite3['PalletQuantity']==0])

213

In [37]:
# excluded other zero pallet quantity orders; these are mostly sample orders
kite3 = kite3.loc[kite3['PalletQuantity']!=0, :]


In [38]:
kite3['Pallet_PercentageUse'] = kite3['Packs'] * kite3['UnitsPerPack']/kite3['PalletQuantity']
kite3.sort_values(by='Pallet_PercentageUse', ascending=False)

,level_0,index,WebsiteID,DeliveryPostCode,DeliveryCountry,OrderID,OrderDate,StatusName,StatusId,PaymentOptionName,PaymentType,DespatchedCourier,ProductsExVAT,DeliveryExVAT,VAT,SubTotal,GrandTotal,DateDespatched,DeliveryCost,PartNumber,Description,Packs,PackPrice,UnitsPerPack,PackWeight,LineWeight,PalletQuantity,CubicMPerUnit,PostCode,PostCode_Label,Order_Date,Date_Despatched,Despatch_Delay,DayOfWeek,DayOfWeek_n,Expected_Delivery,Despatch_Delay_n,Delivery_Date,Pallet_PercentageUse
857,862,5010,148410,B25 8,United Kingdom,2745519,14/06/2024 14:39:07,Dispatched,6,Payment Gateway,2,DX Freight,1797.60,0.00,351.19,1797.60,2107.12,14/06/2024 00:00:00,115.05,VF15EF,Starch based loose fill - 15 cu ft bag,140,12.84,1,2.00,280.00,4,0.376712,B,1,2024-06-14,2024-06-14,0 days,Friday,5,4,0,2024-06-18,35.000000
1052,1059,6022,181941,B19 3,United Kingdom,2732830,04/06/2024 14:54:08,Dispatched,6,On Account,3,MFS,3500.00,0.00,700.00,3500.00,4200.00,25/06/2024 00:00:00,714.58,CDW400X550X150-82,Custom boxes - 400x550x150mm 0201 BC125WT Whit...,2000,1.75,1,0.74,1480.00,140,NaN,B,1,2024-06-04,2024-06-25,21 days,Tuesday,2,2,21,2024-06-25,14.285714
10475,10547,60789,922186,CV13 0,United Kingdom,2747588,17/06/2024 15:45:18,Dispatched,6,"Cheque, BACS or pro-forma",0,MFS,1942.00,19.99,392.40,1942.00,2354.39,17/06/2024 00:00:00,382.46,CDW202020,Double wall boxes - 20x20x20 inches (508x508x5...,100,15.55,10,11.23,1123.00,100,0.012829,CV,0,2024-06-17,2024-06-17,0 days,Monday,1,1,0,2024-06-18,10.000000
3766,3795,21229,583013,NG18 5,United Kingdom,2744970,14/06/2024 10:55:16,Dispatched,6,Payment Gateway,2,MFS,2700.00,19.99,544.00,2700.00,3263.99,14/06/2024 00:00:00,404.59,CDW120909M,Double wall boxes - 12x9x9 inches (305x229x229...,360,7.50,20,5.01,1803.60,720,0.002231,NG,2,2024-06-14,2024-06-14,0 days,Friday,5,3,0,2024-06-17,10.000000
9794,9860,56007,889566,LE2 0,United Kingdom,2744835,14/06/2024 10:00:50,Dispatched,6,Payment Gateway,2,MFS,1425.90,0.00,285.18,1425.90,1711.08,14/06/2024 00:00:00,370.47,CDW161616M,Double wall boxes - 16x16x16 inches (406x406x4...,70,20.37,20,13.22,925.40,140,0.006008,LE,2,2024-06-14,2024-06-14,0 days,Friday,5,4,0,2024-06-18,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,2248,11933,380725,B46 1,United Kingdom,2753435,21/06/2024 15:49:31,Dispatched,6,On Account,3,MFS,445.56,0.00,89.11,445.56,534.67,21/06/2024 00:00:00,34.58,AUTO-RETRACT-BLADES,Auto retract replacement blades - pack of 10,1,3.96,1,0.04,0.04,40000,0.000018,B,1,2024-06-21,2024-06-21,0 days,Friday,5,4,0,2024-06-25,0.000025
8333,8398,48182,826078,LE65 1,United Kingdom,2744992,14/06/2024 11:00:57,Dispatched,6,On Account,3,DX Express,44.39,7.29,10.34,44.39,62.02,14/06/2024 00:00:00,6.88,AUTO-RETRACT-BLADES,Auto retract replacement blades - pack of 10,1,3.96,1,0.04,0.04,40000,0.000018,LE,2,2024-06-14,2024-06-14,0 days,Friday,5,4,0,2024-06-18,0.000025
11403,11482,66732,963083,LE9 9,United Kingdom,2743758,13/06/2024 12:12:09,Dispatched,6,Paypal,1,DX Freight,175.28,0.00,35.06,175.28,210.34,13/06/2024 00:00:00,27.11,AUTO-RETRACT-BLADES,Auto retract replacement blades - pack of 10,1,3.96,1,0.04,0.04,40000,0.000018,LE,2,2024-06-13,2024-06-13,0 days,Thursday,4,4,0,2024-06-17,0.000025
2250,2267,12114,387738,DE1 1,United Kingdom,2755426,24/06/2024 16:44:43,Dispatched,6,Paypal,1,MFS,2050.92,0.00,410.18,2050.92,2461.10,25/06/2024 00:00:00,39.43,HEAVY-DUTY-BLADES,Snap-off replacement blades - pack of 10,1,2.49,1,0.01,0.01,40000,0.000019,DE,2,2024-06-24,2024-06-25,1 days,Monday,1,2,1,2024-06-26,0.000025


In [39]:
kite3.isna().sum()

level_0                   0
index                     0
WebsiteID                 0
DeliveryPostCode          0
DeliveryCountry           0
OrderID                   0
OrderDate                 0
StatusName                0
StatusId                  0
PaymentOptionName         0
PaymentType               0
DespatchedCourier         0
ProductsExVAT             0
DeliveryExVAT             0
VAT                       0
SubTotal                  0
GrandTotal                0
DateDespatched            0
DeliveryCost              0
PartNumber                0
Description               0
Packs                     0
PackPrice                 0
UnitsPerPack              0
PackWeight                0
LineWeight                7
PalletQuantity            0
CubicMPerUnit           569
PostCode                  0
PostCode_Label            0
Order_Date                0
Date_Despatched           0
Despatch_Delay            0
DayOfWeek                 0
DayOfWeek_n               0
Expected_Delivery   

In [40]:
kite3.loc[kite3['LineWeight'].isna(),'LineWeight']=0

In [41]:
kite3.isna().sum()

level_0                   0
index                     0
WebsiteID                 0
DeliveryPostCode          0
DeliveryCountry           0
OrderID                   0
OrderDate                 0
StatusName                0
StatusId                  0
PaymentOptionName         0
PaymentType               0
DespatchedCourier         0
ProductsExVAT             0
DeliveryExVAT             0
VAT                       0
SubTotal                  0
GrandTotal                0
DateDespatched            0
DeliveryCost              0
PartNumber                0
Description               0
Packs                     0
PackPrice                 0
UnitsPerPack              0
PackWeight                0
LineWeight                0
PalletQuantity            0
CubicMPerUnit           569
PostCode                  0
PostCode_Label            0
Order_Date                0
Date_Despatched           0
Despatch_Delay            0
DayOfWeek                 0
DayOfWeek_n               0
Expected_Delivery   

In [42]:
kite3['Weight_Per_Pallet'] = kite3['PackWeight']/kite3['UnitsPerPack']*kite3['PalletQuantity']
kite3['Volume_Per_Pallet'] = kite3['CubicMPerUnit']*kite3['PalletQuantity']

In [43]:
# we excluded product parts with overweight/oversized pallet specifications, along with the rest of those orders
orders_to_delete = kite3.loc[(kite3['Weight_Per_Pallet']>1200) | (kite3['Volume_Per_Pallet']>2.64),'OrderID'].unique()

In [44]:
kite3 = kite3.loc[~kite3['OrderID'].isin(orders_to_delete), :]

In [45]:
kite4 = kite3.drop(['level_0','index','WebsiteID','DeliveryPostCode','DeliveryCountry','OrderDate','StatusName','StatusId','PaymentOptionName'
            ,'PaymentType','DespatchedCourier','SubTotal','GrandTotal','VAT','DateDespatched','PartNumber','Packs',
           'PackPrice','UnitsPerPack','PackWeight','CubicMPerUnit','PalletQuantity'], axis=1)
kite4

,OrderID,ProductsExVAT,DeliveryExVAT,DeliveryCost,Description,LineWeight,PostCode,PostCode_Label,Order_Date,Date_Despatched,Despatch_Delay,DayOfWeek,DayOfWeek_n,Expected_Delivery,Despatch_Delay_n,Delivery_Date,Pallet_PercentageUse,Weight_Per_Pallet,Volume_Per_Pallet
0,2748523,62.16,0.00,6.88,Plain grip seal bags - 7.5x7.5 inches (191x191...,0.99,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2,0,2024-06-20,0.000833,1188.00,2.52000
1,2748523,62.16,0.00,6.88,Plain grip seal bags - 4x5.5 inches (102x140 m...,0.42,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2,0,2024-06-20,0.000463,907.20,2.59200
2,2748523,62.16,0.00,6.88,Plain grip seal bags - 12.75x12.75 inches (324...,2.61,CV,0,2024-06-18,2024-06-18,0 days,Tuesday,2,2,0,2024-06-20,0.002500,1044.00,2.28000
3,2754306,77.00,0.00,6.88,Single wall cardboard boxes - 15x10x10 inches ...,19.24,LE,2,2024-06-24,2024-06-24,0 days,Monday,1,2,0,2024-06-26,0.111111,173.16,1.53000
4,2750321,133.54,0.00,18.64,Double wall boxes - 18x12x12 inches (457x305x3...,10.30,DY,1,2024-06-19,2024-06-19,0 days,Wednesday,3,2,0,2024-06-21,0.035714,288.40,2.48584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12701,2758266,12.77,7.26,6.85,Packaging tape - Kite premium acrylic retail p...,0.86,DY,1,2024-06-26,2024-06-26,0 days,Wednesday,3,2,0,2024-06-28,0.001587,541.80,NaN
12702,2758336,145.98,0.00,6.85,Cross weave tape - 75mmx50m - pack of 6,12.96,MK,3,2024-06-26,2024-06-26,0 days,Wednesday,3,2,0,2024-06-28,0.023438,552.96,1.44384
12703,2758462,282.54,19.99,11.70,Paper mailing bags 190 x 50 x 300 mm - standar...,34.50,LE,2,2024-06-26,2024-06-26,0 days,Wednesday,3,1,0,2024-06-27,0.150000,230.00,1.16000
12704,2758538,148.22,0.00,11.07,Foam roll - 2.5mmx1200mmx120m,16.56,NG,2,2024-06-26,2024-06-26,0 days,Wednesday,3,2,0,2024-06-28,0.500000,33.12,1.48840


In [46]:
kite4.isna().sum()

OrderID                   0
ProductsExVAT             0
DeliveryExVAT             0
DeliveryCost              0
Description               0
LineWeight                0
PostCode                  0
PostCode_Label            0
Order_Date                0
Date_Despatched           0
Despatch_Delay            0
DayOfWeek                 0
DayOfWeek_n               0
Expected_Delivery         0
Despatch_Delay_n          0
Delivery_Date             0
Pallet_PercentageUse      0
Weight_Per_Pallet         0
Volume_Per_Pallet       423
dtype: int64

In [47]:
pd.options.display.max_rows = None

In [48]:
grouped_orders = kite4.groupby(['OrderID','Delivery_Date','PostCode','PostCode_Label','DeliveryCost','ProductsExVAT'])\
[['LineWeight','Pallet_PercentageUse']].sum().sort_values(by='Delivery_Date').reset_index()
grouped_orders

,OrderID,Delivery_Date,PostCode,PostCode_Label,DeliveryCost,ProductsExVAT,LineWeight,Pallet_PercentageUse
0,2729774,2024-06-04,NG,2,25.33,351.07,54.28,0.397727
1,2731497,2024-06-04,DE,2,6.95,48.90,14.27,0.126587
2,2731427,2024-06-04,MK,3,84.36,469.52,210.65,1.618386
3,2731398,2024-06-04,OX,4,7.28,68.00,20.40,0.050000
4,2731391,2024-06-04,DE,2,84.36,469.92,332.60,2.000000
5,2731333,2024-06-04,CV,0,13.52,131.52,40.00,0.062500
6,2731298,2024-06-04,WS,1,14.04,205.77,26.51,0.625000
7,2730026,2024-06-04,OX,4,11.22,53.38,9.00,0.500000
8,2731246,2024-06-04,OX,4,44.21,483.56,143.00,0.733333
9,2730388,2024-06-04,WR,5,84.36,520.80,335.44,2.000000


In [49]:
grouped_orders['DeliveryCost'].sum()

95571.35999999999

In [50]:
grouped_orders.groupby(['Delivery_Date','PostCode_Label'])[['DeliveryCost']].sum()

DeliveryCost
Delivery_Date PostCode_Label              
2024-06-04    0                      20.47
              1                     120.03
              2                     156.08
              3                     137.05
              4                     228.87
              5                      84.36
2024-06-05    0                     855.06
              1                    1947.29
              2                    1997.76
              3                    1314.93
              4                     427.01
              5                     192.32
2024-06-06    0                     494.87
              1                    1302.43
              2                    1496.44
              3                     693.20
              4                     425.75
              5                     272.99
2024-06-07    0                     613.08
              1                    1665.07
              2                    1658.26
              3                     919.48
              4                     801.51
              5                     195.16
2024-06-08    2                      37.13
2024-06-10    0                     545.92
              1                    1497.95
              2                    1129.38
              3                     390.35
              4                     178.33
              5                     327.89
2024-06-11    0                     966.14
              1                    1362.06
              2                    1298.01
              3                     602.21
              4                     568.42
              5                     141.08
2024-06-12    0                     870.48
              1                    1736.83
              2                    1721.15
              3                     774.99
              4                     703.52
              5                      78.27
2024-06-13    0                     344.01
              1                    1685.38
              2                    1936.60
              3                     698.77
              4                     167.78
              5                      84.32
2024-06-14    0                     552.06
              1                    1514.76
              2                    1801.23
              3                     876.53
              4                     448.11
              5                     204.11
2024-06-17    0                     493.70
              1                    1784.12
              2                    1627.69
              3                     421.29
              4                     408.29
              5                      98.04
2024-06-18    0                     467.14
              1                    1533.85
              2                    1970.64
              3                     952.97
              4                     309.61
              5                     173.17
2024-06-19    0                     988.17
              1                    1811.67
              2                    2333.43
              3                     942.80
              4                     329.27
              5                     253.78
2024-06-20    0                     555.23
              1                    1467.28
              2                    1692.79
              3                     578.77
              4                     513.93
              5                      46.07
2024-06-21    0                     527.31
              1                    1712.76
              2                    1575.56
              3                     600.98
              4                     397.99
              5                     154.55
2024-06-22    2                     180.10
2024-06-24    0                     461.39
              1                     841.13
              2                    1142.32
              3                     543.97
              4                     620.10
              5                

In [51]:
# Pallet percentage is sorted from small to large which corresponds to the way of loading trucks, ie small orders
# go together to maximize the capacity of trucks and also due to their ease of unloading can make more deliveries.

vehicle = kite4.groupby(['Delivery_Date','PostCode_Label','OrderID'])[['LineWeight','Pallet_PercentageUse']].sum()\
.reset_index().sort_values(['Delivery_Date','PostCode_Label','Pallet_PercentageUse'])
vehicle

,Delivery_Date,PostCode_Label,OrderID,LineWeight,Pallet_PercentageUse
0,2024-06-04,0,2730647,8.72,0.016667
1,2024-06-04,0,2731333,40.00,0.062500
4,2024-06-04,1,2730464,30.72,0.055556
5,2024-06-04,1,2731074,19.65,0.113889
2,2024-06-04,1,2729905,27.38,0.299524
3,2024-06-04,1,2730452,159.24,0.303175
7,2024-06-04,1,2731298,26.51,0.625000
6,2024-06-04,1,2731287,136.43,1.000000
11,2024-06-04,2,2731497,14.27,0.126587
9,2024-06-04,2,2729978,89.08,0.393939


In [52]:
grouped_delivery = kite4.groupby(['Delivery_Date','PostCode_Label'])[['LineWeight','Pallet_PercentageUse']].sum().reset_index()
grouped_delivery

,Delivery_Date,PostCode_Label,LineWeight,Pallet_PercentageUse
0,2024-06-04,0,48.72,0.079167
1,2024-06-04,1,399.93,2.397143
2,2024-06-04,2,490.23,2.918254
3,2024-06-04,3,324.32,3.416770
4,2024-06-04,4,442.40,4.408333
5,2024-06-04,5,335.44,2.000000
6,2024-06-05,0,3757.78,21.337656
7,2024-06-05,1,8496.77,48.389159
8,2024-06-05,2,5956.12,44.789417
9,2024-06-05,3,3737.46,30.018743


In [53]:
grouped_delivery.loc[:,'7.5 ton'] =0

In [54]:
# determine the number of 7.5 ton lorries needed to deliver orders on each day on each route

B = vehicle['Delivery_Date'].unique()
for b in B:
    C = vehicle.loc[vehicle['Delivery_Date']==b,'PostCode_Label'].unique()
    for c in C:
        a=vehicle.loc[(vehicle['Delivery_Date']==b)&(vehicle['PostCode_Label']==c),['LineWeight','Pallet_PercentageUse']]
        x=0
        y=0
        for i in range(len(a)):
            if (x+a.iloc[i,1]<= 14) & (y+a.iloc[i,0]<= 4000):
                x=x+a.iloc[i,1]
                y=y+a.iloc[i,0]
            elif (x+a.iloc[i,1]> 14) & (y+a.iloc[i,0]<= 4000):
                grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                =grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                +math.trunc(x/14)+1
                x=a.iloc[i,1]
                y=a.iloc[i,0]
            elif (x+a.iloc[i,1]<= 14) & (y+a.iloc[i,0]> 4000):
                grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                =grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                +math.trunc(y/4000)+1
                x=a.iloc[i,1]
                y=a.iloc[i,0]
            else:
                grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                =grouped_delivery.loc[(grouped_delivery['Delivery_Date']==b)&(grouped_delivery['PostCode_Label']==c),'7.5 ton']\
                +max(math.trunc(y/4000),math.trunc(x/14))+1
                x=a.iloc[i,1]
                y=a.iloc[i,0]

grouped_delivery['7.5 ton']=grouped_delivery['7.5 ton']+1
grouped_delivery






,Delivery_Date,PostCode_Label,LineWeight,Pallet_PercentageUse,7.5 ton
0,2024-06-04,0,48.72,0.079167,1
1,2024-06-04,1,399.93,2.397143,1
2,2024-06-04,2,490.23,2.918254,1
3,2024-06-04,3,324.32,3.416770,1
4,2024-06-04,4,442.40,4.408333,1
5,2024-06-04,5,335.44,2.000000,1
6,2024-06-05,0,3757.78,21.337656,2
7,2024-06-05,1,8496.77,48.389159,4
8,2024-06-05,2,5956.12,44.789417,4
9,2024-06-05,3,3737.46,30.018743,3


In [55]:
grouped_delivery.groupby('Delivery_Date')['7.5 ton'].sum()

Delivery_Date
2024-06-04     6
2024-06-05    15
2024-06-06    10
2024-06-07    13
2024-06-08     1
2024-06-10     9
2024-06-11     9
2024-06-12    13
2024-06-13    11
2024-06-14    12
2024-06-17    11
2024-06-18    12
2024-06-19    13
2024-06-20    10
2024-06-21     9
2024-06-22     1
2024-06-24     9
2024-06-25    11
2024-06-26    13
2024-06-27    10
2024-06-28    10
2024-06-29     1
Name: 7.5 ton, dtype: int64

In [56]:
# monthly cost of keeping and running 15 lorries

(1250+3200)*15

66750

In [57]:
# compared against the total cost of outsourcing during the month

grouped_orders['DeliveryCost'].sum()

95571.35999999999

In [58]:
kite4.groupby(['Delivery_Date'])[['LineWeight','Pallet_PercentageUse']].sum()

,LineWeight,Pallet_PercentageUse
Delivery_Date,,
2024-06-04,2041.04,15.219667
2024-06-05,23829.15,154.422944
2024-06-06,11868.21,94.692385
2024-06-07,22880.00,123.425491
2024-06-08,18.00,0.156250
2024-06-10,13072.46,81.762066
2024-06-11,14256.10,89.239214
2024-06-12,19230.31,126.678776
2024-06-13,14305.10,100.148283


The delivery demand on peak days can be potentially smoothed out by moving standard deliveries forward or backward to days of lighter demand, including after considering the spiking effect of orders from previous month delayed into June.

In [60]:
kite3['Date_Despatched'].apply(weekday).value_counts()

Date_Despatched
Tuesday      2260
Wednesday    2006
Monday       1989
Thursday     1626
Friday       1199
Sunday         21
Name: count, dtype: int64

The next step would be to add back the entries we deleted because of erroneous and missing values as much as possible, and include the CubicMPerUnit criteria for loading vehicles.